<a href="https://colab.research.google.com/github/Linux-Server/AI_Engineering/blob/main/alpace_120steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
#@title Setup
!pip install trl
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=None
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",


)

Unsloth 2025.10.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
### load the dataset
from datasets import load_dataset

data = load_dataset("vicgalle/alpaca-gpt4", split="train")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-6ef3991c06080e(…):   0%|          | 0.00/48.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [5]:
import pandas as pd

pd.DataFrame(data)

,instruction,input,output,text
0,Give three tips for staying healthy.,,1. Eat a balanced and nutritious diet: Make su...,Below is an instruction that describes a task....
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye...",Below is an instruction that describes a task....
2,Describe the structure of an atom.,,An atom is the basic building block of all mat...,Below is an instruction that describes a task....
3,How can we reduce air pollution?,,There are several ways to reduce air pollution...,Below is an instruction that describes a task....
4,Describe a time when you had to make a difficu...,,"As an AI assistant, I do not have my own perso...",Below is an instruction that describes a task....
...,...,...,...,...
51997,Generate an example of what a resume should li...,,**John Doe**\n\n**Contact Information:**\n\n12...,Below is an instruction that describes a task....
51998,Arrange the items given below in the order to ...,"cake, me, eating",Me eating cake.,"Below is an instruction that describes a task,..."
51999,Write an introductory paragraph about a famous...,Michelle Obama,"Michelle Obama, born January 17, 1964, in Chic...","Below is an instruction that describes a task,..."
52000,Generate a list of five things one should keep...,,1. Evaluate your reasons: It's important to ta...,Below is an instruction that describes a task....


In [6]:
## convert this to share gpt style
from unsloth import to_sharegpt

dataset = to_sharegpt(
    data,
    merged_prompt="{instruction}[[\nThe inputs are : \n{input}]]",
    output_column_name="output",
    conversation_extension=2
)

Merging columns:   0%|          | 0/52002 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/52002 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [7]:
dataset[5]

{'conversations': [{'from': 'human',
   'value': 'Identify the odd one out.\nThe inputs are : \nTwitter, Instagram, Telegram'},
  {'from': 'gpt',
   'value': 'The odd one out is Telegram. Twitter and Instagram are social media platforms mainly for sharing information, images and videos while Telegram is a cloud-based instant messaging and voice-over-IP service.'},
  {'from': 'human',
   'value': 'Paraphrase this statement in your own words.\nThe inputs are : \nAI is playing an increasingly important role in our society.'},
  {'from': 'gpt',
   'value': 'The significance of AI within our community is constantly growing.'}]}

In [8]:
from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [9]:
dataset[5]

{'conversations': [{'content': 'Identify the odd one out.\nThe inputs are : \nTwitter, Instagram, Telegram',
   'role': 'user'},
  {'content': 'The odd one out is Telegram. Twitter and Instagram are social media platforms mainly for sharing information, images and videos while Telegram is a cloud-based instant messaging and voice-over-IP service.',
   'role': 'assistant'},
  {'content': 'Paraphrase this statement in your own words.\nThe inputs are : \nAI is playing an increasingly important role in our society.',
   'role': 'user'},
  {'content': 'The significance of AI within our community is constantly growing.',
   'role': 'assistant'}]}

In [10]:
### setup chat template
from unsloth import apply_chat_template

chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""


chat_template_one = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""


dataset = apply_chat_template(
    dataset,
    tokenizer,
    chat_template_one

)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [11]:
dataset[5]

{'conversations': [{'content': 'Identify the odd one out.\nThe inputs are : \nTwitter, Instagram, Telegram',
   'role': 'user'},
  {'content': 'The odd one out is Telegram. Twitter and Instagram are social media platforms mainly for sharing information, images and videos while Telegram is a cloud-based instant messaging and voice-over-IP service.',
   'role': 'assistant'},
  {'content': 'Paraphrase this statement in your own words.\nThe inputs are : \nAI is playing an increasingly important role in our society.',
   'role': 'user'},
  {'content': 'The significance of AI within our community is constantly growing.',
   'role': 'assistant'}],
 'text': '<|begin_of_text|>Below are some instructions that describe some tasks. Write responses that appropriately complete each request.\nUSER: Identify the odd one out.\nThe inputs are : \nTwitter, Instagram, Telegram\nASSISTANT: The odd one out is Telegram. Twitter and Instagram are social media platforms mainly for sharing information, images a

In [12]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        # num_train_epochs = 1,
        max_steps=120,
        learning_rate = 2e-4,
        logging_steps=10,
        report_to="wandb",
        output_dir="output-120-steps",
        optim="adamw_8bit",
        warmup_steps=12,
        lr_scheduler_type="linear"

    )

)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 52,002 | Num Epochs = 1 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sachin6624 (sachin6624-axomium-labs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.619900
20,1.222100
30,1.083200
40,1.057600
50,1.085500
60,1.021600
70,1.041400
80,1.022100
90,1.043700
100,1.049200


In [36]:
from transformers import TextStreamer

prompt = [{"role": "system", "content": "Your are Sachin Murali and AI model created by Axomium Labs"},
    {"role": "user", "content": "Who are you?"}]


inputs = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

model_inputs = tokenizer(inputs, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer, skip_prompt=True)



_ = model.generate(**model_inputs, max_new_tokens=100, streamer=streamer)


 I am Sachin Murali, a professional AI model created by Axomium Labs.<|end_of_text|>


In [37]:
model.save_pretrained("unmerged_model", tokenizer)

In [39]:
model.save_pretrained_merged("merges", tokenizer, save_method="merged_16bit")

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:57<00:00, 14.26s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:00<00:00, 15.08s/it]


Unsloth: Merge process complete. Saved to `/content/merges`


In [41]:
model.push_to_hub_merged("sachin6624/llama-8b-alpaca-120steps", tokenizer, save_method="merged_16bit")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...a-120steps/tokenizer.json:  14%|#4        | 2.44MB / 17.2MB            

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0004-of-00004.safetensors:   4%|3         | 41.9MB / 1.17GB            

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [04:44<00:00, 71.21s/it]


Unsloth: Merge process complete. Saved to `/content/sachin6624/llama-8b-alpaca-120steps`
